In [ ]:
!pip install torch transformers datasets accelerate peft


In [ ]:
import torch
import math
import pandas as pd
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM, EarlyStoppingCallback
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from huggingface_hub import login
from transformers import Trainer, TrainingArguments

In [ ]:
hugging_face_token = os.getenv("HF_TOKEN")
login(hugging_face_token)

In [ ]:
# Load the Mistral 7B model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Read the data from the .txt file
with open('final_document.txt', 'r') as file:
    lines = file.readlines()

def extract_questions_answers(lines):
    questions = []
    answers = []

    # Initialize a variable to keep track of the last question
    current_question = None

    for line in lines:
        line = line.strip()  # Remove any leading/trailing whitespace
        if line.startswith("Question:"):
            current_question = line.replace("Question: ", "").strip()
        elif line.startswith("Answer:"):
            if current_question:  # Ensure we have a current question
                answer = line.replace("Answer: ", "").strip()
                questions.append(current_question)
                answers.append(answer)
                current_question = None  # Reset for the next pair

    return questions, answers
questions, answers = extract_questions_answers(lines)

In [ ]:
data_frame = pd.DataFrame({"questions": questions, "answers": answers})
dataset = Dataset.from_dict({"questions": questions, "answers": answers})
train_test_split = dataset.train_test_split(test_size=0.1)  # 10% for testing
# Access the training and testing datasets
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
def tokenize_function(examples):
    # Check if the tokenizer has a pad token; if not, set it
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding


    # Combine questions with an instruction
    encodings = tokenizer(
        [f"As a legal assistant, answer the question with references to eu ai act if possible: {q}:" for q in examples['questions']],  # Instruction added
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust as needed
        return_tensors='pt'
    )

    # Tokenize answers as labels
    answer_encodings = tokenizer(
        examples['answers'],
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust as needed
        return_tensors='pt'
    )

    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': answer_encodings['input_ids'],  # Answers are the labels
    }

In [ ]:
# Tokenize the datasetrain_datasetts
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
# Remove the unwanted columns (questions and answers)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['questions', 'answers'])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(['questions', 'answers'])
# Ensure proper formatting (adjust columns as necessary)
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/1793 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
len(tokenized_eval_dataset['input_ids'])

200

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=16,  # Start with 8, can increase if necessary
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "ffn.up_proj", "ffn.down_proj"],
    lora_alpha=32,  # Start with 16, increase to 32 if underfitting
    lora_dropout=0.1,  # Small dropout to prevent overfitting
    task_type="causal"  # Suitable for Q&A or language generation tasks
)


In [ ]:
# Prepare the model with LoRA
model = get_peft_model(model, lora_config)

# Move model to device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Global Parameters
L_RATE = 3e-5
BATCH_SIZE = 2
gradient_accumulation_steps = 2
WEIGHT_DECAY = 0.01
EVAL_STEPS = 50

# Assuming you have already tokenized datasets
print(f"Size of train dataset: {len(tokenized_train_dataset)}")
print(f"Size of eval dataset: {len(tokenized_eval_dataset)}")

# Calculate training steps and epochs
STEPS = int(len(tokenized_train_dataset) / (BATCH_SIZE * gradient_accumulation_steps))
number_of_epochs = 2
MAX_STEPS = int(number_of_epochs * STEPS)
WARM_UP_STEPS = int(0.01 * MAX_STEPS)
LOGGING_STEPS = 50
SAVE_STEPS = 200
SAVE_TOTAL_LIM  = 8

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    push_to_hub=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    max_steps=MAX_STEPS,
    warmup_steps=WARM_UP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    load_best_model_at_end=False,  # No need to load best model since no evaluation
    remove_unused_columns=False,
)

# Initialize Trainer without eval_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Only the training dataset
    # eval_dataset=tokenized_eval_dataset,  # Removed since no evaluation
    # No compute_metrics function needed since there's no evaluation
)


# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Size of train dataset: 1793
Size of eval dataset: 200


Step,Training Loss
50,3.879400
100,1.711300
150,1.695200
200,1.639800
250,1.653400
300,1.644700
350,1.472300
400,1.688900
450,1.446000
500,1.500200


TrainOutput(global_step=896, training_loss=1.6974041291645594, metrics={'train_runtime': 3167.5777, 'train_samples_per_second': 1.131, 'train_steps_per_second': 0.283, 'total_flos': 7.837073987862528e+16, 'train_loss': 1.6974041291645594, 'epoch': 1.9977703455964324})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
output_dir = "/content/eu_ai_model_mistral7b"
trainer.save_model(output_dir)

In [ ]:
trainer.push_to_hub(output_dir)

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1728055052.946e4b288798.13749.0:   0%|          | 0.00/8.83k [00:00<?, ?B/s]

events.out.tfevents.1728053566.946e4b288798.6964.0:   0%|          | 0.00/6.62k [00:00<?, ?B/s]

events.out.tfevents.1728053184.946e4b288798.4607.0:   0%|          | 0.00/5.33k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Orcawise/results/commit/c37ec801d526afd810e5f6162e4c7fa274faf2cb', commit_message='/content/eu_ai_model_mistral7b', commit_description='', oid='c37ec801d526afd810e5f6162e4c7fa274faf2cb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!zip -r saved_model_896_checkpoint.zip results/checkpoint-896

  adding: results/checkpoint-896/ (stored 0%)
  adding: results/checkpoint-896/README.md (deflated 66%)
  adding: results/checkpoint-896/training_args.bin (deflated 51%)
  adding: results/checkpoint-896/optimizer.pt (deflated 8%)
  adding: results/checkpoint-896/scheduler.pt (deflated 56%)
  adding: results/checkpoint-896/adapter_model.safetensors (deflated 7%)
  adding: results/checkpoint-896/rng_state.pth (deflated 25%)
  adding: results/checkpoint-896/adapter_config.json (deflated 54%)
  adding: results/checkpoint-896/trainer_state.json (deflated 71%)


In [ ]:
import os
import time

def generate_response(prompt, max_length=512):
    """Generate response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    """
    With lora config one has to use **kwargs i.e key arguments such as input_ids=inputs["input_ids"]
    """
    outputs = model.generate(
                              input_ids=inputs["input_ids"],
                               max_length=max_length,
                              num_beams=5,
                              repetition_penalty=1.5,
                              temperature=.5,
                              top_k=30,
                              top_p=0.5,
                              early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def real_time_testing(*args):
    """
    Process either a single question or a text file of questions. For a single question, print the response.
    For a text file, save the questions and generated answers to a text file.

    Args:
        *args: Single question as a string or text file path as a string.
    """
    questions = []
    is_single_question = False

    # Check if the input is a single question or a file path
    if len(args) == 1 and isinstance(args[0], str):
        input_arg = args[0]
        if os.path.isfile(input_arg):  # Check if the input is a file path
            txt_filename = input_arg
            # Read questions from text file
            try:
                with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                    questions = [line.strip() for line in txt_file.readlines() if line.strip()]
            except IOError as e:
                print(f"Error reading text file {txt_filename}: {e}")
                return
        else:  # Assume the input is a single question
            questions.append(input_arg)
            is_single_question = True
    else:
        print("Invalid input. Please provide a single question or a text file path.")
        return

    # Initialize variables to hold instruction-output pairs
    instruction_output_pairs = []

    start = time.time()

    # Generate outputs for each question and collect pairs
    for idx, question in enumerate(questions):
        generated_text = generate_response(question)
        instruction_output_pairs.append((question, generated_text))
        if is_single_question:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")
        else:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

    if not is_single_question:
        # Define the output text file path
        output_txt_file = 'generated_outputs.txt'

        # Write instruction-output pairs to text file
        try:
            with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
                for pair in instruction_output_pairs:
                    txt_file.write(f"Question: {pair[0]}\nAnswer: {pair[1]}\n\n")
            print(f"Text file with generated outputs saved: {output_txt_file}")
        except IOError as e:
            print(f"Error writing to text file {output_txt_file}: {e}")

In [ ]:
real_time_testing('Questions_5.txt')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected be

*******************
1. Prompt: What human oversight measures must be described? 
Answer: What human oversight measures must be described?



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
2. Prompt: How should transparency information be communicated to ensure it is actionable and meaningful? 
Answer: How should transparency information be communicated to ensure it is actionable and meaningful?

Transparency information can be communicated in a variety of ways, such as through data visualisations, infographics, or text-based explanations. The key is to ensure that the information is presented in a way that is clear, concise, and easy to understand. It is also important to ensure that the information is presented in a way that is actionable, meaning that users are able to take action based on the information provided. For example, if the information is presented in a way that allows users to compare different products or services, they will be able to make more informed decisions.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

*******************
4. Prompt: How should the EU AI Act approach the regulation of AI-powered autonomous vehicles and other emerging technologies? 
Answer: How should the EU AI Act approach the regulation of AI-powered autonomous vehicles and other emerging technologies?

AI-powered autonomous vehicles (AVs) have the potential to improve road safety, reduce traffic congestion, and increase accessibility for people with disabilities. However, AVs also raise a number of ethical and regulatory challenges.

The EU AI Act is an important piece of legislation that aims to regulate the development and deployment of AI-powered autonomous vehicles. The Act sets out a number of requirements that must be met in order to ensure that AVs are safe, secure, and ethical.

One of the key requirements of the EU AI Act is that AVs must be designed in a way that minimises the risk of accidents. This means that AVs must be able to detect and avoid obstacles, and they must be able to respond appropriately t

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
5. Prompt: How effective will the EU AI Act be in regulating AI development outside the EU? 
Answer: How effective will the EU AI Act be in regulating AI development outside the EU?

The EU’s AI Act, which is set to come into force in 2024, aims to regulate the development and use of artificial intelligence (AI) systems within the European Union. However, it remains to be seen how effective the Act will be in regulating AI development outside the EU.

There are several factors that could impact the effectiveness of the EU’s AI Act in regulating AI development outside the EU. Firstly, there is the issue of enforcement. The Act relies on EU member states to enforce its provisions, and it is unclear how effectively they will be able to do so. Secondly, there is the issue of compliance. The Act requires AI systems to comply with certain requirements, such as transparency and accountability. However, it is unclear whether AI developers outside the EU will be willing or a

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
6. Prompt: How does the EU AI Act define "high-risk AI systems" and what are their specific transparency obligations? 
Answer: How does the EU AI Act define "high-risk AI systems" and what are their specific transparency obligations?

The EU AI Act defines high-risk AI systems as AI systems that pose a risk to the health, safety, or fundamental rights of individuals. These systems must comply with specific transparency obligations, such as providing clear information about the AI system's capabilities, limitations, and potential risks.

How does the EU AI Act define "high-risk AI systems"

*******************
7. Prompt: How does the AI Act address bias and fairness in AI systems? 
Answer: How does the AI Act address bias and fairness in AI systems?

Time taken: 54.947317361831665 seconds
Text file with generated outputs saved: generated_outputs.txt
